# FUEL

In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, DateType,TimestampType

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark import SQLContext

import json 

from datetime import datetime, timedelta
import math
import os
from IPython.core.display import display, HTML
from datetime import datetime
import pyspark.conf as conf
conf.autoBroadcastJoinThreshold = -1 # this parameters avoids stopping the program when it takes too long
display(HTML("<style>.container { width:100% !important; }</style>"))

from elasticsearch import Elasticsearch
import urllib3

urllib3.disable_warnings()

C:\Users\Maria\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
spark = SparkSession.builder.appName("TFG").getOrCreate()
elastic = Elasticsearch(hosts=['http://127.0.0.1:9200/'])

# SCHEMAS

In [3]:
schema_vehicle = StructType([ \
    StructField("DEVICE_ID",StringType(),True), \
    StructField("LATITUDE",DoubleType(),True), \
    StructField("LONGITUDE",DoubleType(),True), \
    StructField("DATETIME",TimestampType(),True), \
    StructField("SPEED",DoubleType(),True), \
  ])
schema_fuel = StructType([ \
    StructField("DEVICE_ID",StringType(),True), \
    StructField("LITRES",DoubleType(),True), \
    StructField("DATETIME",TimestampType(),True)
  ])

# First look at the data

In [4]:
first_look_csv = 'data/fuel_original/fuel_20210201_20210201_0_anon.csv' 

first_look_csv = spark.read.csv(first_look_csv, header=True, inferSchema=True, sep='|', schema=schema_fuel)

first_look_csv.printSchema()
first_look_csv.show(3, truncate=False)

root
 |-- DEVICE_ID: string (nullable = true)
 |-- LITRES: double (nullable = true)
 |-- DATETIME: timestamp (nullable = true)

+---------+------------------+-----------------------+
|DEVICE_ID|LITRES            |DATETIME               |
+---------+------------------+-----------------------+
|4BF9LOXT |2413.4900000000002|2021-02-01 01:00:18.147|
|QI9IXIGJ |696.38            |2021-02-01 01:00:05.02 |
|QI9IXIGJ |696.38            |2021-02-01 01:00:06.023|
+---------+------------------+-----------------------+
only showing top 3 rows



# COMMON FUNCTIONS

In [5]:
def get_distance(lat1deg, lon1deg, lat2deg, lon2deg):
    '''
    return: distance in metres from 2 locations (lat, lon)
    '''
    if not all((lat1deg, lon1deg, lat2deg, lon2deg)):
        return 0.0
    
    #approximate radius of earth in m
    R = 6373000.0

    lat1 = math.radians(lat1deg)
    lon1 = math.radians(lon1deg)
    lat2 = math.radians(lat2deg)
    lon2 = math.radians(lon2deg)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

# register as a UDF 
get_distance_udf = F.udf(get_distance, DoubleType())

In [6]:
def get_day_month_year(filename):
    '''
    return: day, month and year of the filename
    '''
    dia = filename.split('_')[1][6:]
    mes = filename.split('_')[1][4:6]
    ano = filename.split('_')[1][:4]
    
    return dia, mes, ano

In [7]:
def get_delta_time(timestr): 
    '''
    return: deltatime as float in seconds from HH:MM:SS.mmm
    '''
    values = timestr.split(':')
    #Generate a timedelta
    delta = timedelta(hours=float(values[0]), minutes=float(values[1]), seconds=float(values[2]))
    #Represent in Seconds
    return delta.total_seconds()

# register as a UDF 
get_delta_time_udf = F.udf(get_delta_time, DoubleType())

In [8]:
def abs_time_delta(y,x): 
    '''
    return: duration in seconds
    '''
    if not all((x, y)):
        return 0.0
    delta = math.fabs((x-y).total_seconds())
    return delta

# register as a UDF 
abs_time_delta_udf = F.udf(abs_time_delta, DoubleType())

In [9]:
# return time - 31 seconds
mintimeudf = F.udf(lambda x: (x - timedelta(seconds=31)), TimestampType())
# return time + 31 seconds
maxtimeudf = F.udf(lambda x: (x + timedelta(seconds=31)), TimestampType())

# FUEL TREATMENT

In [10]:
def get_fuel(file, df_vehicle):
    df_fuel_initial = spark.read.csv(file, header=True,inferSchema=True, sep='|',schema=schema_fuel
                                    ).withColumnRenamed('DATETIME', 'DATETIME_F')
    df_fuel1 = df_fuel_initial.withColumn('prev_fuel', F.lag('LITRES').over(Window.partitionBy("DEVICE_ID").orderBy('DATETIME_F'))) \
                        .withColumn('diff_fuel', F.when(F.isnull(F.col('prev_fuel')),0).otherwise(F.col('LITRES')-F.col('prev_fuel'))) \
                        .withColumn('start_time', mintimeudf(F.col('DATETIME_F'))) \
                        .withColumn('end_time', maxtimeudf(F.col('DATETIME_F'))) \
                        .drop('prev_fuel')
    
    
    df_fuel2 = df_fuel1.join(df_vehicle, on=((df_vehicle.DEVICE_ID==df_fuel1.DEVICE_ID) &
                                         (df_vehicle.DATETIME_V.between(df_fuel1.start_time, df_fuel1.end_time))), how='left') \
                    .drop('start_time', 'end_time') \
                    .drop(df_vehicle.DEVICE_ID) \
                    .cache()
    #df_fuel_no_date = df_fuel2.where(F.col('DATETIME_V').isNull())
    df_fuel3 = df_fuel2.where(F.col('DATETIME_V').isNotNull())
    window_fuel = Window.partitionBy(["DEVICE_ID", "DATETIME_F"]).orderBy(F.col('diff_date').desc())
    df_fuel4 = df_fuel3.withColumn('diff_date', abs_time_delta_udf(F.col('DATETIME_F'), F.col('DATETIME_V'))) \
                    .withColumn('next_diff_date', F.lead('diff_date', default=1000).over(window_fuel)) \
                    .withColumn('result', F.when(F.col('diff_date') < F.col('next_diff_date'), 1).otherwise(0)) \
                    .filter(F.col('result') == 1) \
                    .drop('diff_date', 'next_diff_date', 'result')
    df_fuel5 = df_fuel4.withColumn('PREV_LATITUDE', F.lag('LATITUDE').over(Window.partitionBy("DEVICE_ID").orderBy('DATETIME_F'))) \
                    .withColumn('PREV_LONGITUDE', F.lag('LONGITUDE').over(Window.partitionBy("DEVICE_ID").orderBy('DATETIME_F'))) \
                    .withColumn('DISTANCE', get_distance_udf(F.col('LATITUDE'), F.col('LONGITUDE'), F.col('PREV_LATITUDE'), F.col('PREV_LONGITUDE'))) \
                    .select('DEVICE_ID','LITRES','DATETIME_F','diff_fuel','SPEED','DISTANCE') \
                    .cache()
    
    return df_fuel5

     

In [11]:
MAIN_START_TIME = datetime.now()
directory = "data/fleet_original/"
starting_time = datetime.now()
for filename in os.listdir(directory):
    # leemos csv
    if filename.endswith(".csv"):
        
        dia, mes, ano =  get_day_month_year(filename)
        
        #---------------------
        df_vehicles = spark.read.csv(directory+filename, header=True,inferSchema=True, sep='|',schema=schema_vehicle
                                    ).withColumnRenamed('DATETIME', 'DATETIME_V')
        print('{} read at {}'.format(filename, datetime.now()))

        #---------------------
        output_file = open('json_data/fuel_ES/fuel_ES_{}{}{}.json'.format(ano,mes,dia,ano,mes,dia), "wt")

        
        directory_fuel = "data/fuel_original/"
        fuel_filename = 'fuel_{}{}{}_{}{}{}_0_anon.csv'.format(ano,mes,dia,ano,mes,dia)
        df_fuel = get_fuel(directory_fuel + fuel_filename, df_vehicles)
        
        df_diff = df_fuel.where(F.col('diff_fuel')>0).groupBy('DEVICE_ID').agg(F.sum('diff_fuel').alias('total_diff_fuel'))
        df_dist = df_fuel.groupBy('DEVICE_ID').agg(F.sum('DISTANCE').alias('total_distance'))
        
        
        data_fuel = df_diff.join(df_dist, 'DEVICE_ID', 'inner')
        
        print('df_fuel loaded at {}'.format(datetime.now()))
        data_fuel = data_fuel.rdd.toLocalIterator()
        
        for row in data_fuel:
            date = '{}/{}/{}'.format(ano, mes, dia)
            print('device_id = {} \ndate={} \ndiff_fuel={} \ndistance={} '.format(row['DEVICE_ID'],date,row['total_diff_fuel'],row['total_distance']))
            elastic.index(index='fuel_index',
                             document={'device_id': "'" + row['DEVICE_ID'] + "'",
                                       'date': date, 
                                       'diff_fuel': row['total_diff_fuel'],
                                       'distance': row['total_distance']})
            
            
print('fuel_treatment process finished (duration = {} hours, {} minutes)'.format(
    ((datetime.now() - MAIN_START_TIME).seconds)//3600,
    (((datetime.now() - MAIN_START_TIME).seconds)//60)%60))

logrecords_20210201_20210201_0_anon.csv read at 2022-05-06 22:23:14.675138
df_fuel loaded at 2022-05-06 22:23:16.400543
device_id = 4XUR6E8S 
date=2021/02/01 
diff_fuel=2.75 
distance=35825.368660156775 
device_id = 73TO3F88 
date=2021/02/01 
diff_fuel=1.92999999999995 
distance=17375.410829366618 
device_id = 9UHI5H8P 
date=2021/02/01 
diff_fuel=2.599999999999909 
distance=31058.30409310515 
device_id = CFHZ38OO 
date=2021/02/01 
diff_fuel=10.220000000000255 
distance=146170.33741989115 
device_id = EIRCXU5I 
date=2021/02/01 
diff_fuel=1.5899999999999181 
distance=15967.35955349592 
device_id = YHI8RFLK 
date=2021/02/01 
diff_fuel=2.040000000000191 
distance=27871.91919052066 
device_id = IFEMQIQA 
date=2021/02/01 
diff_fuel=1.6300000000001091 
distance=15379.252313050256 
device_id = SLMLB1US 
date=2021/02/01 
diff_fuel=5.490000000000009 
distance=58625.643230611044 
device_id = 6OTUZKR5 
date=2021/02/01 
diff_fuel=4.869999999999891 
distance=52283.7824209282 
device_id = GHKWPH6E 
d

device_id = CWTMTQF9 
date=2021/02/01 
diff_fuel=0.42000000000007276 
distance=2126.2554171693046 
device_id = 1IJFT994 
date=2021/02/01 
diff_fuel=4.409999999999968 
distance=81164.0424536683 
device_id = 63WHUDWM 
date=2021/02/01 
diff_fuel=4.360000000000127 
distance=70275.72211567005 
device_id = CKQRFO76 
date=2021/02/01 
diff_fuel=11.060000000000173 
distance=196057.42051206063 
device_id = BB91OP9D 
date=2021/02/01 
diff_fuel=6.670000000000073 
distance=96813.36660761022 
device_id = ZUDX7QRF 
date=2021/02/01 
diff_fuel=5.589999999999918 
distance=58795.609040053525 
device_id = HLD4OC85 
date=2021/02/01 
diff_fuel=2.6800000000000637 
distance=48753.955750742185 
device_id = IT2DVT4H 
date=2021/02/01 
diff_fuel=2.4600000000000364 
distance=12396.690234974545 
device_id = MC34JZPE 
date=2021/02/01 
diff_fuel=3.2899999999999636 
distance=39551.594469753865 
device_id = DX9ZDX7U 
date=2021/02/01 
diff_fuel=8.0300000000002 
distance=160845.19990748007 
device_id = QMXVLAG0 
date=202

device_id = 10XPA79L 
date=2021/02/01 
diff_fuel=9.159999999999854 
distance=138494.1546762852 
device_id = 2X1V8LFB 
date=2021/02/01 
diff_fuel=6.870000000000118 
distance=105630.66513573914 
device_id = 6G7TEFKA 
date=2021/02/01 
diff_fuel=1.1100000000000136 
distance=12561.85852755398 
device_id = 7WJSLWAK 
date=2021/02/01 
diff_fuel=0.8899999999998727 
distance=6530.163010921583 
device_id = 7XUW89WI 
date=2021/02/01 
diff_fuel=3.259999999999991 
distance=49267.93577467306 
device_id = LITY30QJ 
date=2021/02/01 
diff_fuel=2.240000000000009 
distance=21929.069534498536 
device_id = X9YVBBH0 
date=2021/02/01 
diff_fuel=4.550000000000182 
distance=45194.803423908466 
device_id = 02X0FJBH 
date=2021/02/01 
diff_fuel=4.129999999999882 
distance=41593.69484416589 
device_id = 5OBUCFTP 
date=2021/02/01 
diff_fuel=3.75 
distance=49754.8162881537 
device_id = 9O658CMC 
date=2021/02/01 
diff_fuel=2.8299999999999272 
distance=52180.55676098246 
device_id = GT42SJ4M 
date=2021/02/01 
diff_fuel

device_id = ROUN10M1 
date=2021/02/01 
diff_fuel=9.620000000000005 
distance=104694.78428965359 
device_id = URV0HDMZ 
date=2021/02/01 
diff_fuel=0.6100000000001273 
distance=8247.601997265363 
device_id = VK9SP7R1 
date=2021/02/01 
diff_fuel=4.3599999999999 
distance=61193.45629898798 
device_id = CQMY4N5P 
date=2021/02/01 
diff_fuel=1.1499999999999773 
distance=6589.4968140985375 
device_id = HVZLZQLJ 
date=2021/02/01 
diff_fuel=0.6200000000000045 
distance=4770.782489426339 
device_id = 1HXVQLLP 
date=2021/02/01 
diff_fuel=15.559999999999945 
distance=162511.08066014547 
device_id = 7M9SKDO0 
date=2021/02/01 
diff_fuel=3.3099999999999454 
distance=34922.66590607937 
device_id = GYMIQXC0 
date=2021/02/01 
diff_fuel=4.5 
distance=42892.337196140455 
device_id = M22KSNKT 
date=2021/02/01 
diff_fuel=14.159999999999854 
distance=184474.1970586729 
device_id = U4IDEFXV 
date=2021/02/01 
diff_fuel=0.09999999999990905 
distance=174.09891097239333 
device_id = 30G14PCE 
date=2021/02/01 
diff

device_id = HE6O8RO3 
date=2021/02/01 
diff_fuel=4.459999999999582 
distance=51930.10140108561 
device_id = JJ7C6Z77 
date=2021/02/01 
diff_fuel=8.370000000000005 
distance=119586.83846009345 
device_id = M3ISI50M 
date=2021/02/01 
diff_fuel=5.309999999999945 
distance=83703.80561046024 
device_id = UFP86HFE 
date=2021/02/01 
diff_fuel=4.120000000000346 
distance=47107.02446126422 
device_id = G6TLEL8Y 
date=2021/02/01 
diff_fuel=2.2999999999999545 
distance=49398.57522191861 
device_id = GIMQ11HF 
date=2021/02/01 
diff_fuel=9.079999999999927 
distance=90332.53544179068 
device_id = JD655SOM 
date=2021/02/01 
diff_fuel=5.269999999999982 
distance=43316.73002704246 
device_id = QAV6UU4A 
date=2021/02/01 
diff_fuel=10.55999999999949 
distance=154379.93171150316 
device_id = Y4ERYKI5 
date=2021/02/01 
diff_fuel=4.0 
distance=34582.15301855367 
device_id = YGFES4OC 
date=2021/02/01 
diff_fuel=0.23999999999978172 
distance=582.9327290019963 
device_id = F8WDJH6H 
date=2021/02/01 
diff_fuel=

...

device_id = BYT33L8V 
date=2021/02/26 
diff_fuel=2.660000000000082 
distance=62805.405119358446 
device_id = HHUCHA8P 
date=2021/02/26 
diff_fuel=1.2199999999998 
distance=14490.897860672656 
device_id = YA2BG5NL 
date=2021/02/26 
diff_fuel=0.6100000000001273 
distance=4684.50060166566 
device_id = HR9DM942 
date=2021/02/26 
diff_fuel=4.1200000000000045 
distance=40229.65363400262 
device_id = PWQWX4HD 
date=2021/02/26 
diff_fuel=5.919999999999959 
distance=62055.56619178014 
device_id = PYDIUPDM 
date=2021/02/26 
diff_fuel=9.859999999999673 
distance=154869.71386090864 
device_id = UTGBU9KM 
date=2021/02/26 
diff_fuel=0.6400000000003274 
distance=6261.065696397219 
device_id = WJIPKNL4 
date=2021/02/26 
diff_fuel=3.4699999999999704 
distance=58343.273714364834 
device_id = CQPOQ6T6 
date=2021/02/26 
diff_fuel=7.079999999999927 
distance=100554.82205835497 
device_id = X5AUL3WG 
date=2021/02/26 
diff_fuel=4.920000000000073 
distance=54890.19262249024 
device_id = 2YKVDDNZ 
date=2021/02

device_id = IFKABDZ9 
date=2021/02/26 
diff_fuel=2.119999999999891 
distance=25224.409738357364 
device_id = 3MRG8JYU 
date=2021/02/26 
diff_fuel=13.720000000000027 
distance=152216.719553876 
device_id = N8OGIC63 
date=2021/02/26 
diff_fuel=2.6000000000000227 
distance=29918.900263889016 
device_id = RAQEH7UG 
date=2021/02/26 
diff_fuel=0.5499999999997272 
distance=4791.645773667647 
device_id = TH5QNDZQ 
date=2021/02/26 
diff_fuel=0.8899999999998727 
distance=7297.3364106313875 
device_id = 2H7G2PDR 
date=2021/02/26 
diff_fuel=3.1299999999996544 
distance=25498.50248427692 
device_id = 2JNAQE7L 
date=2021/02/26 
diff_fuel=0.9199999999999591 
distance=6770.5685733526125 
device_id = EOEI7HH2 
date=2021/02/26 
diff_fuel=2.9500000000000455 
distance=40777.30950429061 
device_id = GHLX2FEC 
date=2021/02/26 
diff_fuel=8.180000000000291 
distance=96564.29257133753 
device_id = HR3BIBGO 
date=2021/02/26 
diff_fuel=1.8099999999999739 
distance=18441.661311281885 
device_id = K17CYXDP 
date=2

device_id = M0DUKXZB 
date=2021/02/26 
diff_fuel=4.9900000000002365 
distance=70002.16300992155 
device_id = MYBIB2QQ 
date=2021/02/26 
diff_fuel=4.759999999999991 
distance=64592.24220322121 
device_id = O3AO690X 
date=2021/02/26 
diff_fuel=1.819999999999709 
distance=14617.838196361427 
device_id = TZCCGQ5T 
date=2021/02/26 
diff_fuel=2.2200000000000273 
distance=27561.659937136967 
device_id = 6DEO2M4H 
date=2021/02/26 
diff_fuel=9.490000000000236 
distance=99407.73575563991 
device_id = A8OYPXJA 
date=2021/02/26 
diff_fuel=5.580000000000041 
distance=96028.87110712282 
device_id = GX9I6P2V 
date=2021/02/26 
diff_fuel=12.150000000000091 
distance=169613.70323954796 
device_id = XOPKQ3WZ 
date=2021/02/26 
diff_fuel=6.849999999999454 
distance=92041.00593291967 
device_id = Z3DW7LFD 
date=2021/02/26 
diff_fuel=0.010000000000218279 
distance=0.0 
device_id = 0PIFCIEB 
date=2021/02/26 
diff_fuel=0.9900000000002365 
distance=14709.1397710134 
device_id = 5EWN66C8 
date=2021/02/26 
diff_f

device_id = M41S05OE 
date=2021/02/26 
diff_fuel=4.110000000000127 
distance=58258.54338947444 
device_id = RLS0GKXQ 
date=2021/02/26 
diff_fuel=7.829999999999927 
distance=115004.74026565273 
device_id = F47GSFAN 
date=2021/02/26 
diff_fuel=3.8299999999999272 
distance=40165.623799860405 
device_id = MBO4NBC1 
date=2021/02/26 
diff_fuel=2.3600000000001273 
distance=27802.24250815225 
device_id = NOLVJIRV 
date=2021/02/26 
diff_fuel=2.7900000000000773 
distance=27069.274768912986 
device_id = Q169JF21 
date=2021/02/26 
diff_fuel=2.3100000000004 
distance=19467.844505329795 
device_id = VEK6JX9J 
date=2021/02/26 
diff_fuel=0.8000000000000682 
distance=9729.724583715317 
device_id = VH1LVS1V 
date=2021/02/26 
diff_fuel=2.6399999999998727 
distance=29640.3213020307 
device_id = 22JOJYFB 
date=2021/02/26 
diff_fuel=11.909999999999968 
distance=152947.3534434754 
device_id = 7FY1AGM7 
date=2021/02/26 
diff_fuel=4.0900000000001455 
distance=43873.722286575554 
device_id = FN0YOULG 
date=2021

device_id = LQ9BMDBV 
date=2021/02/26 
diff_fuel=3.3900000000003274 
distance=25988.69347120158 
device_id = QFEBMW3Z 
date=2021/02/26 
diff_fuel=0.020000000000436557 
distance=33.61659401422392 
device_id = Y7DW1GAU 
date=2021/02/26 
diff_fuel=13.930000000000064 
distance=165943.23270348768 
device_id = 00MJTFIG 
date=2021/02/26 
diff_fuel=10.210000000000036 
distance=164011.59358474266 
device_id = 081KQY4G 
date=2021/02/26 
diff_fuel=0.7699999999999818 
distance=7242.335581077686 
device_id = 2MF9MIS2 
date=2021/02/26 
diff_fuel=12.510000000000218 
distance=140437.89835306356 
device_id = 7DH6D8KG 
date=2021/02/26 
diff_fuel=7.990000000000009 
distance=129111.65193819375 
device_id = 7DZH19WM 
date=2021/02/26 
diff_fuel=8.309999999999945 
distance=48806.83285612539 
device_id = KMSCT75U 
date=2021/02/26 
diff_fuel=1.2100000000000364 
distance=15684.045524957062 
device_id = QHAIEA48 
date=2021/02/26 
diff_fuel=4.5 
distance=62102.08368401507 
device_id = VOCR3WVT 
date=2021/02/26 
d